In [2]:
!pip install rocketpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.5/372.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.6 MB/s eta 0:00:00
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=6a39f9dc5a394f6ec02702dd2957e006aadf5f2b986afb7fdd63e1d49373f44d
  Stored in directory: /root/.cache/pip/wheels/72/3e/80/c3e5c354c3cbe62d8c5e4fb63d9e7cdccc7f93399997ae465f
Successfully built simplekml


In [4]:
help(Environment)

Help on class Environment in module rocketpy.environment.environment:

class Environment(builtins.object)
 |  Environment(gravity=None, date=None, latitude=0.0, longitude=0.0, elevation=0.0, datum='SIRGAS2000', timezone='UTC', max_expected_height=80000.0)
 |  
 |  Keeps all environment information stored, such as wind and temperature
 |  conditions, as well as gravity.
 |  
 |  Attributes
 |  ----------
 |  Environment.earth_radius : float
 |      Value of Earth's Radius as 6.3781e6 m.
 |  Environment.air_gas_constant : float
 |      Value of Air's Gas Constant as 287.05287 J/K/Kg
 |  Environment.gravity : Function
 |      Gravitational acceleration. Positive values point the
 |      acceleration down. See :meth:`Environment.set_gravity_model` for more
 |      information.
 |  Environment.latitude : float
 |      Launch site latitude.
 |  Environment.longitude : float
 |      Launch site longitude.
 |  Environment.datum : string
 |      The desired reference ellipsoid model, the follow

In [10]:
# 1. Define the Environment
env = Environment(

    latitude=32.990254,
    longitude=-106.974998,
    elevation = 1400,
    date = (2025,5,20,12)
)
  # (year, month, day, hour UTC)
# Corrected method to set the atmospheric model from forecast
env.setAtmosphericModelFromForecast(file="GFS")
env.info()

# 2. Define the Solid Motor
motor = SolidMotor(
    thrustSource="data/motor.thrust",  # Replace with your thrust curve file or array
    burnOut=3.9,
    grainNumber=5,
    grainSeparation=5/1000,
    grainDensity=1815,
    grainOuterRadius=33/1000,
    grainInitialInnerRadius=15/1000,
    grainInitialHeight=120/1000,
    nozzleRadius=33/1000,
    throatRadius=11/1000,
    interpolationMethod="linear"
)
motor.info()

# 3. Define the Rocket
rocket = Rocket(
    motor=motor,
    radius=127/2000,
    mass=19.197 - 2.956,
    inertiaI=6.60,
    inertiaZ=0.0351,
    distanceRocketNozzle=-1.255,
    distanceRocketPropellant=-0.85704,
    powerOffDrag=0.5,
    powerOnDrag=0.5
)
rocket.setRailButtons([0.2, -0.5])

# Add aerodynamic surfaces
rocket.addNose(length=0.55829, kind="vonKarman", distanceToCM=1.278)
rocket.addFinSet(
    n=4, span=0.100, rootChord=0.120, tipChord=0.040, distanceToCM=-1.04956
)
rocket.addTail(topRadius=0.0635, bottomRadius=0.0435, length=0.060, distanceToCM=-1.194656)

# Add sensors and electronics
rocket.addParachute(
    'main',
    CdS=10.0,
    trigger='apogee',
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5)
)
rocket.addParachute(
    'drogue',
    CdS=1.0,
    trigger='altitude > 800 and verticalVelocity < 0',
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5)
)

rocket.allInfo()

# 4. Define Flight Simulation
flight = Flight(rocket, environment=env, inclination=90, heading=0)
flight.allInfo()

AttributeError: 'Environment' object has no attribute 'setAtmosphericModelFromForecast'